# 🔬 Protein Engineering with ProteinMPNN: Sequence Generation + Evaluation

This notebook provides a full pipeline for **protein sequence optimization** using [Neurosnap](https://neurosnap.ai)’s API suite. Starting from a PDB structure, it leverages **ProteinMPNN** to generate optimized sequences under custom constraints—then evaluates them using state-of-the-art predictors for **stability**, **solubility**, and **structure**.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NeurosnapInc/neurosnap/blob/main/example_notebooks/protein_optimization.ipynb)

---

## 📝 Instructions

1. **Set up your environment**
   Ensure your Python environment includes the required packages listed below.

2. **Get your API Key**
   Generate a secure API key at: [neurosnap.ai/overview?view=api](https://neurosnap.ai/overview?view=api)
   > ⚠️ **Important:** Never share your API key with untrusted notebooks or third parties.
   >
   > A typical key looks like this:
   > **fd9f9be1974aa6f1609430c1e477926d4884188d5f752b5071457e10440b816773b92c0f1116442e54364734fd141537fcb6ce1619ad6825679f86511f38a80e**

3. **Upload a protein structure**
   Use a `.pdb` file as your starting point.

4. **Configure optimization settings**
   Set custom parameters like temperature, mutation mode, or positional constraints.

5. **Run the notebook**
   You’ll be prompted for:

   * Your API key
   * The uploaded structure

6. **Evaluate the sequences**
   Each generated sequence is scored by:

   * **TemStaPro** for thermostability
   * **NetSolP** for solubility
   * **Boltz-1** for structure prediction

7. **Review the results**
   Outputs are formatted for downstream use, with sequence names, ranks, and scores.

8. **Cleanup (Optional)**
   You may delete your API key after the run.

---

## ⚡ Why It’s Useful

* **Free to use** — no usage-based billing, credits on Neurosnap available separately
* **Saves time** — skip manual sequence formatting
* **Integrated tools** — unified predictions in one pipeline
* **Modular design** — extend with mutation rules, docking, or epitope tools

---

## 📦 Dependencies

Install using pip:

```bash
pip install git+https://github.com/NeurosnapInc/neurosnap.git ipywidgets tqdm pandas biopython
```

---

## 👏 Credits

Written by Bhat Mohsin
Improved by Keaun Amani

In [ ]:
# @title 🔧 Install Dependencies
# @markdown Run this code cell to install all the dependencies for this notebook.

# @markdown **NOTE:** This cell only needs to be executed once.
import os

os.system("pip install git+https://github.com/NeurosnapInc/neurosnap.git ipywidgets tqdm pandas biopython")

In [ ]:
# @title 🔧 Configure Notebook
# @markdown Set your inputs and preferences below, then run this cell.
# @markdown After running, upload the PDB file of the protein or enzyme you want to optimize.
# @markdown You can also adjust optional parameters — like the number of sequences, temperature, or amino acid biases — to fine-tune the sequence generation to your specific needs.
# @markdown
# @markdown ---

import json

import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from IPython.display import display

from neurosnap.api import NeurosnapAPI
from neurosnap.log import logger

# @markdown ### Notebook Settings
# @markdown
API_KEY = ""  # @param {type:"string", placeholder:"Enter your neurosnap API key"}
api = NeurosnapAPI(api_key=API_KEY)

# @markdown ---
# @markdown ### Inverse Folding Settings
homo_oligomer = False  # @param {type:"boolean"}
invert_selection = False  # @param {type:"boolean"}
number_sequences = 100  # @param {type:"integer"}
sampling_temperature = 0.1  # @param {type:"number"}
model_type = "v_48_020"  # @param ["v_48_020", "v_48_010", "v_48_002"] {type:"string"}
model_version = "original"  # @param ["original", "multimer"] {type:"string"}
# @markdown #### Amino Acid Biases
# @markdown Set bias values between -5 and 5 for each amino acid
alanine_bias = 0  # @param {type:"integer"}
arginine_bias = 0  # @param {type:"integer"}
asparagine_bias = 0  # @param {type:"integer"}
aspartic_acid_bias = 0  # @param {type:"integer"}
cysteine_bias = 0  # @param {type:"integer"}
glutamine_bias = 0  # @param {type:"integer"}
glutamic_acid_bias = 0  # @param {type:"integer"}
glycine_bias = 0  # @param {type:"integer"}
histidine_bias = 0  # @param {type:"integer"}
isoleucine_bias = 0  # @param {type:"integer"}
leucine_bias = 0  # @param {type:"integer"}
lysine_bias = 0  # @param {type:"integer"}
methionine_bias = 0  # @param {type:"integer"}
phenylalanine_bias = 0  # @param {type:"integer"}
proline_bias = 0  # @param {type:"integer"}
serine_bias = 0  # @param {type:"integer"}
threonine_bias = 0  # @param {type:"integer"}
tryptophan_bias = 0  # @param {type:"integer"}
tyrosine_bias = 0  # @param {type:"integer"}
valine_bias = 0  # @param {type:"integer"}

# Upload the file via UI
uploaded = files.upload()
original_filename = list(uploaded.keys())[0]
new_filename = "my_uploaded_file.pdb"
os.rename(original_filename, new_filename)
# Use the renamed file path
file_path = new_filename

# submit job
fields = {
  "Input Structure": ("structure.pdb", open(file_path, "rb")),
  "Number Sequences": str(number_sequences),
  "Sampling Temperature": str(sampling_temperature),
  "Model Type": model_type,
  "Model Version": model_version,
  "Alanine Bias": str(alanine_bias),
  "Arginine Bias": str(arginine_bias),
  "Asparagine Bias": str(asparagine_bias),
  "Aspartic acid Bias": str(aspartic_acid_bias),
  "Cysteine Bias": str(cysteine_bias),
  "Glutamine Bias": str(glutamine_bias),
  "Glutamic acid Bias": str(glutamic_acid_bias),
  "Glycine Bias": str(glycine_bias),
  "Histidine Bias": str(histidine_bias),
  "Isoleucine Bias": str(isoleucine_bias),
  "Leucine Bias": str(leucine_bias),
  "Lysine Bias": str(lysine_bias),
  "Methionine Bias": str(methionine_bias),
  "Phenylalanine Bias": str(phenylalanine_bias),
  "Proline Bias": str(proline_bias),
  "Serine Bias": str(serine_bias),
  "Threonine Bias": str(threonine_bias),
  "Tryptophan Bias": str(tryptophan_bias),
  "Tyrosine Bias": str(tyrosine_bias),
  "Valine Bias": str(valine_bias),
}
mpnn_job_id = api.submit_job("ProteinMPNN", fields=fields, note="Protein Optimization Notebook | Initial inverse folding run")

In [ ]:
# @title 📥 Retrieve and Parse ProteinMPNN Output
# @markdown This cell fetches the output from your completed **ProteinMPNN job**, extracts the top-ranked sequences, and stores them with names like `prot1`, `prot2`, etc., where the number corresponds to the sequence’s rank.
# @markdown
# @markdown For example, `prot7` means the sequence ranked 7th in ProteinMPNN's output.
# @markdown
# @markdown You can control how many sequences to keep for downstream tools (e.g., **TemStaPro** or **NetSolP**) by adjusting the `N` parameter below.

# wait for job
status = api.wait_job_status(mpnn_job_id)
assert status == "completed", f"ProteinMPNN job with ID {mpnn_job_id} failed."

# download results file
api.get_job_file(mpnn_job_id, "out", "results.csv", "proteinmpnn_output.csv")

# Read the CSV data and create a dictionary
proteins = {}
N = 50  # @param {"type":"integer"}
# Open the file
with open("/content/proteinmpnn_output.csv", "r") as file:
  # Skip the header line
  next(file)

  # Process each line
  count = 0
  for line in file:
    # Stop after 50 sequences
    if count >= N:
      break

    # Split the line by comma
    parts = line.strip().split(",")

    # Extract rank and sequence
    rank = int(parts[0])
    sequence = parts[3]

    # Create protein name (prot1, prot2, etc.)
    prot_name = f"prot{rank}"

    # Add to dictionary with prot name as the key
    proteins[prot_name] = sequence

    # Increment counter
    count += 1

# Now proteins dictionary contains only the first N (50) sequences

In [ ]:
# @title 🔥 Submit Sequences to TemStaPro for Thermostability Prediction
# @markdown TemStaPro creates two outputs:
# @markdown
# @markdown mean_output.csv = average stability score
# @markdown output.csv = per residue stability values (can also be accessed via Neurosnap dashboard)

# submit job
fields = {
  "Input Sequences": json.dumps({"aa": proteins, "dna": {}, "rna": {}}),
}
temstapro_job_id = api.submit_job(
  "TemStaPro Protein Thermostability Prediction", fields=fields, note="Protein Optimization Notebook | Thermostability Calculation"
)

# wait for job
status = api.wait_job_status(temstapro_job_id)
assert status == "completed", f"TemStaPro job with ID {temstapro_job_id} failed."

# download results file
api.get_job_file(temstapro_job_id, "out", "mean_output.csv", "Tempstat_output.csv")

In [ ]:
# @title 💧 Submit Sequences to NetSolP for Solubility Prediction
# @markdown This cell sends your ranked ProteinMPNN sequences to **NetSolP** to predict their solubility.
# @markdown
# @markdown Results will be saved to a file named `NetSolp_output.csv` in your working directory.


# submit job
fields = {
  "Input Sequences": json.dumps({"aa": proteins, "dna": {}, "rna": {}}),
}
netsolp_job_id = api.submit_job("NetSolP-1.0", fields=fields, note="Protein Optimization Notebook | Solubility prediction")

# wait for job
status = api.wait_job_status(netsolp_job_id)
assert status == "completed", f"NetSolP-1.0 job with ID {netsolp_job_id} failed."


# download results file
api.get_job_file(netsolp_job_id, "out", "netsol_predictions.csv", "NetSolp_output.csv")

In [ ]:
# @title 💧 View Top Soluble Sequences (Sorted by NetSolP Score)
# @markdown This cell reads the NetSolP output and displays the top protein sequences ranked by **mean solubility**, from highest to lowest.
# @markdown
# @markdown By default, it shows the top **10** sequences. You can change this by modifying the `n` parameter below.
# @markdown
# @markdown The sequences retain their original IDs (e.g., `prot1`, `prot2`) from the ProteinMPNN output to help track them across tools.


def get_top_soluble_proteins(csv_file, n):
  """
  Reads a CSV file of protein data and returns the top N proteins with highest mean solubility.

  Args:
      csv_file (str): Path to the CSV file containing protein data
      n (int): Number of top proteins to return

  Returns:
      DataFrame: Top N proteins sorted by mean solubility in descending order
  """
  # Read the CSV file
  df = pd.read_csv(csv_file)

  # Sort the dataframe by Mean Solubility in descending order
  sorted_df = df.sort_values(by="Mean Solubility", ascending=False)

  # Select only the columns we want to display
  result_df = sorted_df[["ID", "Sequence", "Mean Solubility"]].head(n)

  return result_df


# Set your file path and number of proteins here
csv_file = "/content/NetSolp_output.csv"
n = 10  # @param {"type":"integer"}

# Get the top soluble proteins
top_proteins = get_top_soluble_proteins(csv_file, n)

# Reset the index to avoid displaying it
top_proteins = top_proteins.reset_index(drop=True)

# Configure pandas display options for better visualization
pd.set_option("display.max_colwidth", 50)  # Truncate long sequences for better display
pd.set_option("display.precision", 4)  # Set precision for float values

# Print header
logger.info(f"Top {n} proteins with highest mean solubility:")

# Fix the formatting issues with styling compatible with older pandas versions
styled_table = (
  top_proteins.style.set_properties(
    **{"text-align": "center", "white-space": "pre-wrap", "font-size": "11pt", "border": "1px solid #3d3d3d", "padding": "8px"}
  )
  .set_table_styles(
    [
      {
        "selector": "th",
        "props": [
          ("text-align", "center"),
          ("font-weight", "bold"),
          ("background-color", "black"),
          ("border", "1px solid #CCCCCC"),
          ("padding", "8px"),
        ],
      },
      {"selector": "caption", "props": [("text-align", "center"), ("font-size", "14pt"), ("padding", "10px")]},
      {"selector": "", "props": [("border-collapse", "collapse")]},
      # Hide index using CSS instead of hide_index() method
      {"selector": ".row_heading, .blank", "props": [("display", "none")]},
    ]
  )
  .format({"Mean Solubility": "{:.4f}"})
)

# Display the styled table
display(styled_table)

# Optional: Format sequence display with truncation
# For older pandas versions, we might need to do this before styling
top_proteins["Sequence"] = top_proteins["Sequence"].apply(lambda x: x[:40] + "..." if len(x) > 40 else x)
logger.info("\nWith truncated sequences:")
display(top_proteins)

# Optional: Save the results to a CSV file
# top_proteins.to_csv("top_soluble_proteins.csv", index=False)

In [ ]:
# @title 🔥 Rank Sequences by Thermostability at a Specific Temperature
# @markdown This cell ranks the generated protein sequences based on their **raw thermal stability scores** at a chosen temperature (e.g., 40°C, 45°C, 50°C, 55°C, 60°C, or 65°C).
# @markdown
# @markdown ✅ **Defaults:**
# @markdown - Temperature: **60°C**
# @markdown - Top N sequences shown: **10** (`N_TOP_PROTEINS = 10`)
# @markdown
# @markdown 📌 You can change the `TEMPERATURE` value to target a different condition — for example:
# @markdown ```python
# @markdown TEMPERATURE = 55
# @markdown ```
# @markdown
# @markdown The output includes the `protein_id`, full `sequence`, and the raw value at the selected temperature.

# Configuration
CSV_FILE = "/content/Tempstat_output.csv"
N_TOP_PROTEINS = 10  # @param {"type":"integer"}
TEMPERATURE = 50  # @param {"type":"integer"}

# Read the entire CSV file into a DataFrame
thermostability_data = pd.read_csv(CSV_FILE)

# Determine the column name for the specified temperature
temp_column = f"t{TEMPERATURE}_raw"

# Find matching columns (with more flexible matching)
matching_columns = [col for col in thermostability_data.columns if temp_column.lower() in col.lower()]
if not matching_columns:
  raise ValueError(f"Temperature {TEMPERATURE} is not valid. Choose from 40, 45, 50, 55, 60, or 65.")

# Use the first matching column
column_to_use = matching_columns[0]
logger.info(f"Using column: {column_to_use}")

# Sort and select the top proteins
top_proteins = (
  thermostability_data.sort_values(by=column_to_use, ascending=False)[["protein_id", "sequence", column_to_use]]
  .head(N_TOP_PROTEINS)
  .rename(columns={column_to_use: f"Raw Value (T={TEMPERATURE})"})
)

# Configure pandas display options
pd.set_option("display.max_colwidth", None)  # Show full sequences
pd.set_option("display.precision", 4)  # Set precision for float values

# Display results
logger.info(f"\nTop {N_TOP_PROTEINS} proteins with highest raw values at temperature {TEMPERATURE}°C:\n")
display(top_proteins)

In [ ]:
# @title 🧪 Find Sequences That Are Both Soluble and Thermostable
# @markdown This cell identifies the **best-balanced protein sequences** by combining:
# @markdown - 🔹 **Solubility score** (from NetSolP)
# @markdown - 🔹 **Thermal stability score** at a selected temperature (from TemStaPro)
# @markdown
# @markdown The combined score is calculated by simply **adding** the two values. Sequences are then ranked in descending order of this combined score.
# @markdown
# @markdown 📌 You can customize:
# @markdown - `TEMPERATURE`: The target temperature for thermostability evaluation (e.g., 50, 55, 60)
# @markdown - `TOP_N`: Number of top sequences to display
# @markdown
# @markdown This is especially useful for selecting candidates suited for **real-world conditions**, where both solubility and stability matter.


# Configuration parameters
SOLUBILITY_FILE = "/content/NetSolp_output.csv"
THERMOSTABILITY_FILE = "/content/Tempstat_output.csv"
TEMPERATURE = 50  # @param {"type":"integer"}
TOP_N = 20  # @param {"type":"integer"}

# Load both datasets into DataFrames
solubility_data = pd.read_csv(SOLUBILITY_FILE)
thermostability_data = pd.read_csv(THERMOSTABILITY_FILE)

# Display column names for debugging and inspection
logger.info("Solubility data columns:", solubility_data.columns.tolist())
logger.info("Thermostability data columns:", thermostability_data.columns.tolist())

# Determine temperature column
temp_col = f"t{TEMPERATURE}_raw"
if temp_col not in thermostability_data.columns:
  # Try more flexible matching
  matching_cols = [col for col in thermostability_data.columns if temp_col.lower() in col.lower()]
  if matching_cols:
    temp_col = matching_cols[0]
  else:
    raise ValueError(f"Temperature column for {TEMPERATURE}°C not found in thermostability file")

logger.info(f"Using temperature column: {temp_col}")

# Select and prepare relevant columns
sol_subset = solubility_data.rename(columns={"ID": "protein_id"})[["protein_id", "Mean Solubility"]]
thermo_subset = thermostability_data[["protein_id", temp_col]]

# Merge the two dataframes
merged_data = pd.merge(sol_subset, thermo_subset, on="protein_id", how="inner")

# Calculate combined value
merged_data["combined_value"] = merged_data["Mean Solubility"] + merged_data[temp_col]

# Get top proteins
top_proteins = merged_data.sort_values(by="combined_value", ascending=False).head(TOP_N)

# Display the results
logger.info(f"\nTop {TOP_N} proteins with highest combined solubility and thermostability at {TEMPERATURE}°C:\n")
display(top_proteins)

# Optional: Add a visualization

plt.figure(figsize=(10, 6))
plt.scatter(top_proteins["Mean Solubility"], top_proteins[temp_col], s=100, alpha=0.7, c=top_proteins["combined_value"], cmap="viridis")

for i, row in top_proteins.iterrows():
  plt.annotate(row["protein_id"], (row["Mean Solubility"], row[temp_col]), xytext=(5, 5), textcoords="offset points")

plt.colorbar(label="Combined Value")
plt.xlabel("Solubility Score")
plt.ylabel(f"Thermostability Score (T={TEMPERATURE}°C)")
plt.title(f"Top {TOP_N} Proteins: Solubility vs Thermostability")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# @title 🧬 Predict 3D Structure of a Selected Sequence with Boltz-1 (AlphaFold3)
# @markdown After evaluating solubility and stability, you can use this cell to predict the **3D structure** of any optimized sequence.
# @markdown
# @markdown 👉 Just enter the name (ID) of the protein sequence you want to fold — for example: `prot1`, `prot17`, etc.
# @markdown
# @markdown The selected sequence will be submitted to **Boltz-1 (AlphaFold3)** for structure prediction.
# @markdown
# @markdown 🧠 Once the job completes, you can **view the predicted structure** directly from your [Neurosnap dashboard](https://neurosnap.ai/overview).


def extract_protein_sequence(csv_file, protein_id):
  """
  Extract protein sequence from CSV file for a given protein ID.

  Args:
      csv_file: Path to the CSV file containing protein sequences
      protein_id: ID of the protein to search for

  Returns:
      Protein sequence as string, or None if protein not found
  """
  try:
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Find the row with the matching protein_id
    protein_row = df[df["protein_id"] == protein_id]

    # Check if protein was found
    if protein_row.empty:
      return None

    # Extract the sequence
    sequence = protein_row["sequence"].values[0]

    return sequence

  except Exception as e:
    logger.error(f"Error: {e}")
    return None


# File path
csv_file = "/content/Tempstat_output.csv"

# Protein ID to search for
protein_id = ""  # @param {type:"string", placeholder:"enter a the protein name e,g prot37"}


# Extract sequence
protein_sequence = extract_protein_sequence(csv_file, protein_id)

# submit job
fields = {
  "Input Sequences": json.dumps({"aa": {protein_id: protein_sequence}}),
  # "Input Molecules": json.dumps([{"data": open("receptor.sdf").read(),"type": "sdf"}, {"data": "C=C=C", "type": "smiles"}]),
  # "Residue Modifications": "Protein_1:102:MLY",
  # "Binder Sequence": "Protein_1",
  # "Pocket Restraints": "Protein_2:66",
  # "Covalent Restraints": "Protein_1:6:CA:Protein_2:26:CB",
  # "MSA Mode": "mmseqs2_uniref_env",
  # "Number Recycles": "6",
  # "Sampling Steps": "200",
  # "Diffusion Samples": "5",
  # "Step Scale": "1.638",
}
boltz1_job_id = api.submit_job("Boltz-1 (AlphaFold3)", fields=fields, note="Protein Optimization Notebook | Folding of new structures")